# Backtesting ML Classification-Based

## Load the model

In [47]:
import pickle

In [50]:
with open('models/model_dt_microsoft_linkedin_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [ ]:
model_dt

In [ ]:
model_dt.predict(X=?)

## Load the data

In [1]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Simulated.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,daily_diff,daily_diff_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.882095,UP,1,1
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,0.183765,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,4.649994,UP,1,0
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800,10.760010,UP,1,0


## Backtesting.py Library

### Create your Strategy Class

In [3]:
from backtesting import Backtest, Strategy

In [4]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [5]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800


In [7]:
df.daily_diff_direction[-1:]

Date
2023-03-16    UP
Name: daily_diff_direction, dtype: object

In [40]:
class SimpleClassificationUD(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[-1:, :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [ ]:
bt = Backtest(df_explanatory, SimpleClassificationUD,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [ ]:
results = bt.run()

### Interpret backtesting results

In [45]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-16 00:00:00
Duration,2289 days 00:00:00
Exposure Time [%],99.873177
Equity Final [$],2594.859728
Equity Peak [$],12777.631651
Return [%],-74.051403


### Visualize the strategy performance

In [46]:
bt.plot(filename='reports_backtesting/backtesting_classification.html')

Row(id='8548', ...)